In [1]:
RUNNING_LOCALLY = True

In [13]:
import os

# PLEASE SET YOUR OWN WORKING_DIRECTORY WHEN RUNNING LOCALLY
WORKING_DIRECTORY = "/home/yash/Desktop/Courses/CS2470/Final_Project/working_dir/"

if not RUNNING_LOCALLY:
    os.chdir("/home/yash/")
    print("Current Directory ->", os.getcwd())

    WORKING_DIRECTORY = os.getcwd()

    # Ensure that you are working in the right environment
    !echo $CONDA_PREFIX

In [ ]:
import gym
import time
import numpy as np
from collections import deque
from gym import spaces
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [4]:
class ConcatObs(gym.Wrapper):
    def __init__(self, env, k):
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = \
            spaces.Box(low=0, high=255, shape=((k,) + shp), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        return np.array(self.frames)

In [5]:
# A bunch of wrappers to get us started, please use these
class ObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, GRAYSCALE=False, NORMALIZE=False):
        self.GRAYSCALE = GRAYSCALE
        self.NORMALIZE = NORMALIZE
        super().__init__(env)
    
    def observation(self, obs):
        # Normalise observation by 255
        if self.NORMALIZE:
            obs = obs / 255.0
        # Convert to grayscale -> This isn't quite working right now, but we can update the function quite easily later
        if self.GRAYSCALE:
            obs = np.dot(obs[...,:3], [0.299, 0.587, 0.114])
        return obs

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, reward):
        # Clip reward between 0 to 1
        return np.clip(reward, 0, 1)
    
class ActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def action(self, action):
        return action

In [6]:
env = gym.make("ALE/Riverraid-v5")

A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [7]:
# Using the wrappers for the environment
env = ObservationWrapper(RewardWrapper(ActionWrapper(ConcatObs(env, 4))), GRAYSCALE=False, NORMALIZE=True)
obs = env.reset()

**DQN**

In [35]:
hidden_size = 100
num_actions = 18
# q_net = tf.keras.Sequential()
# q_net.add(tf.keras.layers.Flatten())
# q_net.add(tf.keras.layers.Dense(hidden_size, activation='relu'))
# q_net.add(tf.keras.layers.Dense(num_actions, activation='softmax'))
#we can try to add convolutional layers

def create_q_model(input_shape, hidden_size, num_actions):
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=input_shape)

    # Convolutions on the frames on the screen
    # layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(inputs)
    # layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    # layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)

    layer1 = layers.Flatten()(inputs)
    
    layer2 = tf.keras.layers.Dense(hidden_size, activation='relu')(layer1) 
    action = tf.keras.layers.Dense(num_actions, activation='softmax')(layer2)

    return keras.Model(inputs=inputs, outputs=action)

q_net = create_q_model(input_shape=obs.shape, hidden_size=hidden_size, num_actions=num_actions)

In [36]:
q_net.get_weights()

[array([[ 9.2445360e-04, -5.0038588e-04,  1.8502246e-03, ...,
          3.2054186e-03, -1.0457470e-03,  2.6690653e-03],
        [ 4.5484910e-04,  2.6876340e-03,  1.1422448e-03, ...,
         -2.7786223e-03,  2.6721102e-03, -1.2223334e-03],
        [ 7.2598457e-05,  3.5621896e-03, -2.6043085e-03, ...,
          1.9543828e-03, -2.7319037e-03,  3.7012389e-05],
        ...,
        [-1.6887188e-03, -2.5809696e-03,  1.7139334e-03, ...,
          1.0453900e-03, -5.4468424e-04, -3.0088935e-03],
        [ 3.1211260e-03, -1.3501954e-03,  1.6895821e-05, ...,
          1.5446688e-03, -2.8549910e-03, -3.1436065e-03],
        [-3.9374665e-04, -1.6063054e-03,  1.9851960e-03, ...,
          2.8538830e-03, -2.8794259e-03, -3.0444178e-03]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.

In [19]:
trainer = tf.keras.optimizers.SGD(learning_rate = 0.1)

**One episode**

In [20]:
#hyper parameters
num_steps = 500
obs = env.reset()
rTot = 0
gamma = 0.99
e = 0.2


for step in range(num_steps):

    
    with tf.GradientTape() as tape:
        # Compute the Q values and best action for the current state
        q_values = q_net(np.array([obs]))
        action = tf.math.argmax(q_values,axis=1).numpy()[0]
        
        # Epsilon adaptive
        if np.random.rand(1) < e:
            action = env.action_space.sample()

        # apply the action
        nst_obs, reward, done, _ = env.step(action)
        
        
        q_values_nst = q_net(np.array([nst_obs]))
        action_nst = tf.math.argmax(q_values_nst,axis=1).numpy()[0]
        
        td_error = q_values.numpy()
        td_error[0][action] = reward + gamma*action_nst
        loss = tf.reduce_sum(tf.square(td_error-q_values))
        
    grad = tape.gradient(loss,q_net.trainable_variables)
    trainer.apply_gradients(zip(grad,q_net.trainable_variables))
    rTot += reward
    
    obs = nst_obs
    
    if done:
        break
    


    
# Close the env
env.close()

In [27]:
type(q_net)

keras.engine.functional.Functional

In [31]:
q_net.save(WORKING_DIRECTORY + "model_1_trial")

INFO:tensorflow:Assets written to: /home/yash/Desktop/Courses/CS2470/Final_Project/working_dir/model_1_trial/assets


In [32]:
q_net_copy = keras.models.load_model(WORKING_DIRECTORY + "model_1_trial")

In [37]:
q_net_copy.get_weights()

[array([[-3.0506279e-03,  2.3967847e-03, -2.2423535e-03, ...,
         -1.1952850e-04, -1.7089252e-03, -3.0145252e-03],
        [ 3.2449388e-03,  3.4740660e-04,  1.0411497e-03, ...,
         -9.4713503e-04,  2.6586130e-03,  2.3412569e-03],
        [ 1.9532652e-03,  2.8169909e-03,  1.6335221e-03, ...,
         -3.5981813e-03,  7.2118593e-04, -3.0723370e-03],
        ...,
        [-1.0799554e-03,  3.8140146e-03, -1.9022273e-03, ...,
         -4.8977369e-04, -2.5845249e-03,  1.5689335e-03],
        [-3.0405628e-03, -3.2310425e-03, -3.0168858e-03, ...,
         -2.2355788e-03,  2.0216224e-03, -1.7025322e-03],
        [-2.4454824e-03, -1.8618915e-03, -1.6140898e-03, ...,
         -5.9294142e-04,  2.6996946e-05,  1.0203468e-03]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.

In [33]:
rTot

9.0

**Episodes with n_games**

In [ ]:
#hyperparameters
num_steps = 500
n_games = 15000
E = 1000
rTot = 0
gamma = 0.99
n_games = 15000
obs = env.reset()


for i in range(0, n_games):
    print(f'Running game {i+1}/{n_games}...')
    e = E / (i + E)
    st = game.reset()
    for step in range(num_steps):

    
        with tf.GradientTape() as tape:
            # Compute the Q values and best action for the current state
            q_values = q_net(np.array([obs]))
            action = tf.math.argmax(q_values,axis=1).numpy()[0]

            # Epsilon adaptive
            if np.random.rand(1) < e:
                action = env.action_space.sample()

            # apply the action
            nst_obs, reward, done, _ = env.step(action)


            q_values_nst = q_net(np.array([nst_obs]))
            action_nst = tf.math.argmax(q_values_nst,axis=1).numpy()[0]

            td_error = q_values.numpy()
            td_error[0][action] = reward + gamma*action_nst
            loss = tf.reduce_sum(tf.square(td_error-q_values))

        grad = tape.gradient(loss,q_net.trainable_variables)
        trainer.apply_gradients(zip(grad,q_net.trainable_variables))
        rTot += reward

        obs = nst_obs

        if done:
            break
    
# Close the env
env.close()